# Homework 6: pt 1

In [1]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az
from scipy import stats
import matplotlib.pyplot as plt

In [7]:
#Dataset can be found here: https://www.kaggle.com/datasets/yakhyojon/national-basketball-association-nba
url = 'https://raw.githubusercontent.com/juliacmurray/STA365/main/STA365%20nba-players.csv'
df = pd.read_csv(url, index_col=0)


In [8]:
X = df.filter(items=['min', 'pts', '3p_made']) #predict based on worry about vaccine side effects, worry about h1n1, and if respondent is a healthcare worker
Y = df.filter(items=['target_5yrs']) #whether respondent got vaccinated against h1n1
p = 3 #number of parameters

In [14]:
#Testing OG code
with pm.Model() as MLR:
  betas = pm.MvNormal('betas', mu = np.zeros((p,1)), cov = np.eye(p), shape = (p,1))
  sigma = pm.TruncatedNormal('sigma', mu = 1, sigma = 1, lower = 0)
  y = pm.Normal('y', mu = pm.math.dot(X, betas), sigma = sigma)
  link = pm.math.invlogit(y)
  preds = pm.Bernoulli('preds', p = link, observed = Y)

with MLR:
  idata = pm.sample()

# Pt 2
**Ridge Regression L2 Penalty Derivation (with $b_i = 0$ and $σ = 1$)**
$$posterior of \beta ∝p(\beta | x) * p(\beta)$$
$$= \frac{1}{\sqrt{2π}}e^{\frac{-(y - x^T \beta)^2}{2}} *
\frac{1}{\sqrt{2π}} e^{\frac{-1}{2} (\frac{\beta_i}{s_i})^2}$$
$$∝e^{\frac{-(y - x^T \beta)^2}{2}}e^{\frac{-1}{2} (\frac{\beta_i}{s_i})^2}$$
$$= e^{\frac{-(y - x^T \beta)^2}{2} + \frac{-1}{2} (\frac{\beta_i}{s_i})^2}$$
Taking the negative log, we get:
$$\frac{1}{2} ∑(y - x^T_i \beta)^2 + ∑ \frac{\beta_i^2}{2s_i^2}$$
Take $s_i^2 = \frac{1}{2λ}$:
$$\frac{1}{2} ∑(y - x^T_i \beta)^2 + \lambda ∑ \beta_i^2$$
which is the L2 penalty

**Lasso Regression L1 Penalty Derivation (with $b_i = 0$ and $σ = 1$)**
$$posterior of \beta ∝p(\beta | x) * p(\beta)$$
$$= \frac{1}{\sqrt{2π}}e^{\frac{-(y - x^T \beta)^2}{2}} *
\frac{1}{{2b}} e^{-\frac{|\beta_i|}{s_i}}$$
$$∝e^{\frac{-(y - x^T \beta)^2}{2}}e^{-\frac{|\beta_i|}{s_i}}$$
Taking the negative log, we get:
$$\frac{1}{2} \sum (y - x^T_i \beta)^2 + \sum \frac{| \beta | }{s_i}$$
Take $s_i = 1/λ$:
$$\frac{1}{2} \sum (y - x^T_i \beta)^2 + λ\sum | \beta |$$
which is the L1 penalty
